In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install dash
!pip install dash-bootstrap-components
!pip install jupyter-dash

     |████████████████████████████████| 9.6 MB 2.6 MB/s 
     |████████████████████████████████| 357 kB 63.6 MB/s 
     |████████████████████████████████| 209 kB 3.1 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=4ce971e2d798a7356daae301d7f24dd62492f7270697298eeac70ca3d9ca3bdb
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [1]:
# Importing the libraries
import pickle
import pandas as pd
import webbrowser
import dash
from dash.dependencies import Input, Output, State
from dash import dcc
import dash_bootstrap_components as dbc
from dash import html
import numpy as np
from jupyter_dash import JupyterDash


# Declaring Global variables
print("hello")
app = JupyterDash(__name__, external_stylesheets=[
                dbc.themes.BOOTSTRAP, 'https://use.fontawesome.com/releases/v5.8.1/css/all.css'], suppress_callback_exceptions=True)  # creating object of Dash


# Defining My functions
def load_model():

    global pickle_model
    file = open("/content/drive/My Drive/price_estimation_app/assets/pickle_model.pkl", 'rb')
    pickle_model = pickle.load(file)

    global pickle_cTransformer
    file2 = open("/content/drive/My Drive/price_estimation_app/assets/pickle_cTransformer.pkl", 'rb')
    pickle_cTransformer = pickle.load(file2)


def estimate_price(family_input, total_weight, steel_produced_weight, components_purchased_weight, items, swl):

    to_predict = [family_input, total_weight, steel_produced_weight,
                  components_purchased_weight, items, swl]
    to_predict = np.array(to_predict)
    to_predict = to_predict.reshape(1, 6)
    to_predict = np.array(pickle_cTransformer.transform(
        to_predict), dtype=np.float32)

    return pickle_model.predict(to_predict)

# to automatically open the browser


def open_browser():
    # pass
    webbrowser.open_new('http://127.0.0.1:8030/')


def create_app_ui():
    main_layout = html.Div(
        [
            html.Div([
                html.H4("Winch Details", style={
                        'text-align': 'center', 'color': '#007bff'}),
                html.Hr(),
                html.Div([

                    #html.I("Steel Price EUR"),
                    # html.Br(),
                    #dcc.Input(id="steel-price", type="number", placeholder="",className='input-field'),

                    html.I("Family input"),
                    html.Br(),
                    dcc.Dropdown(
                        id='family-input-dropdown',
                        options=[
                            {'label': x, 'value': x} for x in ['En-labber', 'To-labber', 'To-labber med spoleapparat',
                                                               'To-labber med sandwich', 'A-Frame', 'Moonpool hatch', 'Big winch']
                        ],
                        value=None,
                        placeholder='Select a input...',
                    ),
                    html.Br(),


                    html.I("Total weight"),
                    html.Br(),
                    dcc.Input(id="total-weight", type="number",
                              placeholder="", className='input-field'),

                    html.Br(),
                    html.Br(),

                    html.I("Steel Produced weight"),
                    html.Br(),
                    dcc.Input(id="steel-produced-weight", type="number",
                              placeholder="", className='input-field'),

                    html.Br(),
                    html.Br(),

                    html.I("Number of 160 items"),
                    html.Br(),
                    dcc.Input(id="items", type="number",
                              placeholder="", className='input-field'),

                    html.Br(),
                    html.Br(),

                    html.I("Components purchased weight"),
                    html.Br(),
                    dcc.Input(id="components-purchased-weight", type="number",
                              placeholder="", className='input-field'),

                    html.Br(),
                    html.Br(),

                    html.I("SWL"),
                    html.Br(),
                    dcc.Input(id="swl", type="number",
                              placeholder="", className='input-field'),

                    html.Br(),
                    html.Br(),

                    html.Div([
                        dbc.Row([
                            dbc.Col(html.Div(
                                [

                                    dbc.Button(
                                        children='Estimate',
                                        id='button_estimate',
                                        className='est-btn',
                                        color='primary'
                                    )


                                ]
                            ), md=7),


                            dbc.Col(html.Div(
                                [

                                    dbc.Button(
                                        children='Reset',
                                        id='reset_button',
                                        className='est-btn',
                                        color='primary'
                                    )


                                ]
                            ), md=5)



                        ])
                    ],
                        #style={'text-align': 'center'}
                    ),
                    html.Br(),

                    html.Div([

                        html.H5("Total Price NOK :"),
                        html.H5(id='result', style={'color': '#e25f6b'})
                    ])

                ], className='sub-div')

            ], className='main-div'),
        ]
    )
    return main_layout


@app.callback(
    Output('result', 'children'),
    [
        Input('button_estimate', 'n_clicks')
    ],
    [
        State('family-input-dropdown', 'value'),
        State('total-weight', 'value'),
        State('steel-produced-weight', 'value'),
        State('components-purchased-weight', 'value'),
        State('items', 'value'),
        State('swl', 'value'),
    ]

)
def update_app_ui(n_clicks, family_input, total_weight, steel_produced_weight, components_purchased_weight, items, swl):
    if n_clicks != None:

        if(n_clicks > 0):
            price = estimate_price(
                family_input, total_weight, steel_produced_weight, components_purchased_weight, items, swl)
            print(price, "Estimated price")
            return str(np.round(price[0], 2))


@app.callback(

    [
        Output('family-input-dropdown', 'value'),
        Output('total-weight', 'value'),
        Output('steel-produced-weight', 'value'),
        Output('components-purchased-weight', 'value'),
        Output('items', 'value'),
        Output('swl', 'value'),
        #Output('result', 'children')
    ],
    [Input('reset_button', 'n_clicks')],
    [
        State('family-input-dropdown', 'value'),
        State('total-weight', 'value'),
        State('steel-produced-weight', 'value'),
        State('components-purchased-weight', 'value'),
        State('items', 'value'),
        State('swl', 'value'),
    ]

)
def reset_form(n_clicks, family_input, total_weight, steel_produced_weight, components_purchased_weight, items, swl):
    if n_clicks != None:
        if n_clicks > 0:
            return "", "", "", "", "", ""
        else:
            return "", "", "", "", "", ""

    else:
        return "", "", "", "", "", ""

# Main Function to control the Flow of your Project


def main():
    print("Welcome to Dash app...")
    load_model()

    #open_browser()

    # If you want to assign something to global variable inside a function, then we have to declare that variable as global.
    # If we are just reading, and not assigning then we don't have to declare global

    global project_name
    project_name = "Winch Price Estimation"

    global app
    app.title = project_name
    app.layout = create_app_ui()
    app.run_server(port=8030,mode='inline',host="0.0.0.0")

    print("This would be executed only after the server is down/stopped")

    # Reset all the global variables once the server stops

  




hello


In [ ]:

if __name__ == '__main__':
    main()

Welcome to Dash app...
